In [53]:
import sys
sys.path.append('../')
import os
from libraries.FileManagerKPDS import FileManagerKPDS as fm
from libraries.listdirNoHidden import listdirNoHidden as ldnh
import time
import imagehash
from PIL import Image
import numpy as np

In [54]:
datasetPath = '../../datasets/rescaled_dataset/'
newDatasetPath = '../../datasets/cropped_dataset/'


In [55]:
datasets = ["combined_dataset", "zoomed_dataset", "downscaled_dataset", "gBlur_dataset", "gNoise_dataset", "hor_dataset", "ver_dataset"]
cached_dataset = {}
pathList = ldnh(os.path.join(datasetPath, 'images'))

hashMethods = {"average": imagehash.average_hash, "perceptual": imagehash.phash, "difference": imagehash.dhash, "wavelet": imagehash.whash}
hashMethod = "wavelet"

for imgPath in pathList:
	imgName = os.path.basename(imgPath)
	img1 = Image.open(imgPath)
	hash1 = hashMethods[hashMethod](img1)
	cached_dataset[imgName] = (img1, hash1)


In [56]:
with open('hash'+hashMethod+'1.txt', 'w') as file:
  startTime = time.time()
  totImages = 0
  for dataset in datasets:
    newDatasetPath = '../../datasets/' + dataset + '/'
    newDatasetImagesPaths = ldnh(newDatasetPath)
    totImages += len(newDatasetImagesPaths)
    mapArray = []
    for j, newImgPath in enumerate(newDatasetImagesPaths):
      img2 = Image.open(newImgPath)
      newImgName = newImgPath.split('/')[-1] #sempre precachabile, ma è "solo" O(n^2) (stringhe in python sono lente)
      hash2 = hashMethods[hashMethod](img2)
      scores = [None] * len(cached_dataset)
      for i, dictItem in enumerate(cached_dataset.items()):
        imgName = dictItem[0]
        img1, hash1 = dictItem[1]
        scores[i] = (imgName, hash1 - hash2)
      
      scores.sort(key=lambda tup: tup[1])
      f0 = 0
      combinedTransforms = ""
      if(dataset == "combined_dataset"):
        splits = newImgName.split('_', 1)
        combinedTransforms = splits[0]+"_"
        newImgName = splits[1]
      while(not f0):
        for i in range(len(cached_dataset)):
          s = 1/(i+1)
          if(scores[i][0] == newImgName):
            mapArray.append(s)
            f0 = 1
            break
      file.write(combinedTransforms+newImgName+" "+str(mapArray[-1])+"\n")
      print(combinedTransforms+newImgName+" "+str(mapArray[-1]))
    file.write("MAP: "+ str(np.mean(mapArray))+ " Dataset: "+ dataset+"\n")
    print("MAP: "+ str(np.mean(mapArray))+ " Dataset: "+ dataset+"\n")
  
  stopTime = time.time()

  file.write("Total time: "+ str(stopTime-startTime)+"\n")
  file.write("Time per query: "+ str((stopTime-startTime)/totImages)+"\n")





7GnZoVeGb_20221206_091309.jpg 0.1111111111111111
3VeDo_photo_2022-10-13_16-17-12.jpg 0.023809523809523808
8GbGn_photo_2022-10-13_16-17-12.jpg 1.0
3GnHoGb_photo_2022-10-13_16-16-46.jpg 0.02
1DoZoHo_20221103_080342.jpg 0.018867924528301886
0HoGn_20221206_091312.jpg 1.0
3GnZoVeGb_20221205_122031.jpg 0.2
7DoZoVeGn_20221125_134145.jpg 0.030303030303030304
3ZoHoGb_20221130_131508.jpg 0.034482758620689655
3Zo_20221017_093046.jpg 0.2
9VeGb_20221130_131511.jpg 0.03225806451612903
4GbZoVe_20221107_130316.jpg 0.09090909090909091
4HoZo_20221103_080553.jpg 0.5
0VeZoHo_20221206_091410.jpg 0.04
5HoVeGn_20221017_093241.jpg 0.02564102564102564
6ZoGn_20221130_131502.jpg 0.14285714285714285
8HoZoDoVe_20221206_091413.jpg 0.0196078431372549
8GbHoVe_photo_2022-10-13_16-16-59.jpg 0.021739130434782608
6GbZoVe_20221125_134145.jpg 0.02631578947368421
8Gn_20221108_090912.jpg 1.0
1GbDoHoGn_photo_2022-10-13_16-16-57.jpg 0.043478260869565216
0Ho_20221108_090912.jpg 0.038461538461538464
6ZoDoHoGb_photo_2022-10-13_16